Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


In [115]:
import logging
from collections import namedtuple
import random
from typing import Callable
from copy import deepcopy
from itertools import accumulate
from operator import xor

## The *Nim* and *Nimply* classes

In [116]:
Nimply = namedtuple("Nimply", "row, num_objects")

In [117]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None: #k is the max number of object that can be removed from a line
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    @property
    def k(self) -> int:
        return self._k

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects

## Sample (and silly) startegies 

In [118]:
def nim_sum(state: Nim) -> int:
    *_, result = accumulate(state.rows, xor)
    return result


def cook_status(state: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = [
        (r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1) if state.k is None or o <= state.k
    ]
    cooked["active_rows_number"] = sum(o > 0 for o in state.rows)
    cooked["shortest_row"] = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    cooked["longest_row"] = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    cooked["nim_sum"] = nim_sum(state)    

    brute_force = list()
    for m in cooked["possible_moves"]:
        tmp = deepcopy(state)
        tmp.nimming(m)
        brute_force.append((m, nim_sum(tmp)))
    cooked["brute_force"] = brute_force

    return cooked

In [119]:
def pure_random(state: Nim) -> Nimply:
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)

In [120]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))

In [121]:
def enrico(state: Nim) -> Nimply: #my strategy
    data = cook_status(state)
    if data["active_rows_number"] == 1:
        return (data["longest_row"], state._rows[data["longest_row"]])
    else:
        return (data["shortest_row"], state._rows[data["shortest_row"]])     

In [122]:
def optimal_strategy(state: Nim) -> Nimply:
    data = cook_status(state)
    return next((bf for bf in data["brute_force"] if bf[1] == 0), random.choice(data["brute_force"]))[0]

In [123]:
def hard_coded_strategy(state: Nim, genome) -> Nimply:
    data = cook_status(state)
    aggressive, pref_long, remove_perc = genome
    if data["active_rows_number"] == 1: #win conditions
        return (data["longest_row"], state._rows[data["longest_row"]])
        
    if random.random() > 1-aggressive: #follow an aggressive strategy remove an entire line
        if random.random() > 1-pref_long: 
            return (data["longest_row"], state._rows[data["longest_row"]])
        else:
            return (data["shortest_row"], state._rows[data["shortest_row"]])
    else:   #follow an conservative strattegy remove only a few element in a line
        if random.random() > 1-pref_long:
            num_of_rem = round(state._rows[data["longest_row"]]*remove_perc)
            if num_of_rem == 0: 
                num_of_rem+=1 
            return (data["longest_row"], num_of_rem)
        else:       
            num_of_rem = round(state._rows[data["shortest_row"]]*remove_perc)
            if num_of_rem == 0: 
                num_of_rem+=1 
            return (data["shortest_row"], num_of_rem) 
    return pure_random(state)    
        

In [124]:
def make_strategy(genome: dict) -> Callable:
    def evolvable(state: Nim) -> Nimply:
        data = cook_status(state)

        if random.random() < genome["p"]:
            ply = Nimply(data["shortest_row"], random.randint(1, state.rows[data["shortest_row"]]))
        else:
            ply = Nimply(data["longest_row"], random.randint(1, state.rows[data["longest_row"]]))

        return ply

    return evolvable

In [125]:
NUM_MATCHES = 10
NIM_SIZE = 10


def evaluate(opponent: Callable, genome) -> float:
    won = 0
    last_player_start = 1
    for m in range(NUM_MATCHES):
        nim = Nim(NIM_SIZE)
        player = 1 - last_player_start
        last_player_start = player
        while nim:
            if player == 0:
                ply = opponent(nim)
            else:
                ply = hard_coded_strategy(nim, genome)
            nim.nimming(ply)
            player = 1 - player
        if player == 0:
            won += 1

    return won / NUM_MATCHES #percentage of match won agaist the opponent

In [126]:
evaluate(enrico, (0.5,0.5,0.5)) 

0.8

In [127]:
POPULATION_SIZE = 20
OFFSPRING_SIZE = 10
OPPONENT = gabriele

In [128]:
def fitness(genome): # calcolate the fisness
    return evaluate(OPPONENT, genome)

In [129]:
def generatePopulation(): #return population, one individual is a tuple of a mask array of the list taken and his fitness
    population = list()
    for genome in range(POPULATION_SIZE):
        genome = (random.random(), random.random(), random.random())
        population.append((genome, fitness(genome)))
    return population

In [130]:
def mutation(g):

    point = random.randint(0,len(g)-1)
    return g[:point] + (random.random(),) + g[point+1:]

In [131]:
def select_parent(population, tornament_size=10):
    return max(random.choices(population, k=tornament_size), key=lambda i: i[1])

In [132]:
def GA():
    best_sol = None 
    best_sol_fit = None
    
    population = generatePopulation()
    for generation in range(10):
        offsprings = list()
        for i in range(OFFSPRING_SIZE):
            o = ()
            p = select_parent(population)
            o = mutation(p[0])
            offsprings.append((o, fitness(o)))
        population = population + offsprings   
        population = sorted(population, key=lambda i:i[1], reverse=True)[:POPULATION_SIZE]
            
    best_sol = population[0][0]
    best_sol_fit = population[0][1]
    print("after 100 generations")
    print("best solution:")
    print(best_sol)
    return best_sol

In [133]:
best_sol= GA()
print("win-rate:")
evaluate(gabriele, best_sol)

after 100 generations
best solution:
(0.18118635711206343, 0.03989713934750305, 0.07088704561671222)
win-rate:


0.9